In [ ]:
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb

wget -N https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/118.0.5993.70/linux64/chromedriver-linux64.zip -P /tmp/
unzip -o /tmp/chromedriver-linux64.zip -d /tmp/
chmod +x /tmp/chromedriver-linux64/chromedriver
mv /tmp/chromedriver-linux64/chromedriver /usr/local/bin/chromedriver
pip install selenium chromedriver_autoinstaller

In [ ]:
import json
import requests
import pandas as pd
import numpy as np
import random
from bs4 import BeautifulSoup
import re
import time
import sys
import chromedriver_autoinstaller

sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')


from selenium import webdriver
from selenium.webdriver.chrome.options import Options

from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys


In [ ]:
#--------------------Setting Up Driver---------------------#
def driver_setup():

  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  chrome_options.add_argument("--disable-gpu")

  chromedriver_autoinstaller.install()

  driver = webdriver.Chrome(options=chrome_options)

  return driver

In [ ]:
urls_google_play = ['https://play.google.com/store/apps/details?id=com.google.android.apps.googleassistant',
                    'https://play.google.com/store/apps/details?id=com.openai.chatgpt',
                    'https://play.google.com/store/apps/details?id=ai.replika.app',
                    'https://play.google.com/store/apps/details?id=com.snapchat.android']


#---------------------Element Selector Query & Classes-----------------#
scrollable_class = "odk6He"

#review_button_class = '.VfPpkd-Bz112c-LgbsSe.yHy1rc.eT1oJ.QDwDD.mN1ivc.VxpoF'
review_button_selector = '.VfPpkd-Bz112c-LgbsSe.yHy1rc.eT1oJ.QDwDD.mN1ivc.VxpoF[aria-label="See more information on Ratings and reviews"]'

#star_button_query = '//span[text()="Star rating"]'
star_button_selector = ".odk6He > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(3) > div:nth-child(2)"

star_rating_query = "div.JPdR6b:nth-child(2) > div:nth-child(1) > div:nth-child(1) > span:nth-child({star})"

sort_by_selector = "#sortBy_1"

newest_selector = "div.JPdR6b:nth-child(2) > div:nth-child(1) > div:nth-child(1) > span:nth-child(2)"
#newest_selector = 'div.jO7h3c:contains("Newest")'

In [ ]:
#---------------------Scroll To The End ---------------#
def scroll_to_end(driver, element):
    repeated = 0
    last_height = driver.execute_script("return arguments[0].scrollHeight;", element)

    while True:
        driver.execute_script("arguments[0].scrollIntoView(false);", element)
        time.sleep(1)
        try:
          new_height = driver.execute_script("return arguments[0].scrollHeight;", element)
        except:
          pass
          #new_height = driver.execute_script("return arguments[0].scrollHeight;", element)

        print(f'Currently at {last_height} px. Repeat Count: {repeated}')

        #How far to scroll in pixels...
        if (new_height >3500000):
          print("Pixel height reached 3500000. So Breaking.")
          break

        if (last_height == new_height):
            repeated+=1
            print(f"Reached the end for the {repeated} time.")
            if repeated ==100:
               print(f"The last height was {last_height} and the new height is {new_height}. So breaking.")
               break
        else:
           repeated = 0

        last_height = new_height

In [ ]:
def scrape(url, star=[1,2,3,4,5]):
  global driver
  for star in star:

    app = url.split("id=")[1]
    path = f"/content/drive/MyDrive/HON/data/{app}_{star}_stars.html"

    print(f"Scraping for {path}")
    driver = driver_setup()
    driver.get(url)

    time.sleep(1)
    #Open Reviews&Ratings Popup
    review_button = driver.find_element(By.CSS_SELECTOR, review_button_selector)
    driver.execute_script("arguments[0].click();", review_button)

    #Popup Opens
    time.sleep(1)
    #Click on Star Ratings Filter
    star_button = driver.find_element(By.CSS_SELECTOR, star_button_selector)
    driver.execute_script("arguments[0].click();", star_button)

    time.sleep(1)
    #Click on Specific Star Rating Button
    query = star_rating_query.format(star=star+1)
    star = driver.find_element(By.CSS_SELECTOR, query)
    star.send_keys(Keys.RETURN)


    #Scroll the Scrollable Area
    time.sleep(1)
    scrollable_area = driver.find_element(By.CLASS_NAME, scrollable_class)

    scroll_to_end(driver, scrollable_area)

    #Save the Final Page Source
    content = driver.page_source
    with open(path, 'w', encoding='utf-8') as f:
       f.write(content)

In [ ]:
#Do this for each url in separate colab notebook or using threads for faster scraping

In [ ]:
for url in urls_google_play:
  scrape(url)